In [10]:
from __future__ import division, print_function

import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import pandas as pd
import datetime
import glob

import matplotlib.lines as mlines
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap

%matplotlib inline

In [11]:
### Read the CNV files and extract the variables of interest

#DIRECTORY = globals()['/home/mgrenier/Documents/Data_Arctic/CTDBeaufortGyreWHOI/LSSL_ctd' % i] 
DIRECTORY0 = '/home/mgrenier/Documents/Data_Arctic/CTDBeaufortGyreWHOI/'
DIRECTORY = [DIRECTORY for DIRECTORY in os.listdir(DIRECTORY0) if DIRECTORY.startswith('LSSL')]
DIRECTORY.sort()
#print(DIRECTORY0,DIRECTORY[0])
    ### Range of years over which data were collected
i_init=2003
i_final=2013

press=list(np.chararray((i_final-i_init+1)))
temp=list(np.chararray((i_final-i_init+1)))
sal=list(np.chararray((i_final-i_init+1)))
potemp=list(np.chararray((i_final-i_init+1)))
dens=list(np.chararray((i_final-i_init+1)))

DIR=list(np.chararray((i_final-i_init+1)))


for i in xrange(i_init,i_final+1,1):
    press[i-i_init]='press'+str(i)
    temp[i-i_init]='temp'+str(i)
    sal[i-i_init]='sal'+str(i)
    dens[i-i_init]='dens'+str(i)
    potemp[i-i_init]='potemp'+str(i)
    
    DIR[i-i_init]='DIR'+str(i)

#print(legend)

""""""
for i in xrange(i_init,i_final+1,1):
    ind=0
    print(i)
    DIR[i-i_init] = os.path.join(DIRECTORY0,DIRECTORY[i-i_init])

    QUANTITIES = {
        'prDM': press[i-i_init], # pressure
        't090C': temp[i-i_init], # temperature
        'sal00': sal[i-i_init], # salinity
        'potemp090C': potemp[i-i_init], # potential temperature
        'sigma-é00': dens[i-i_init], # sigma theta
    }
    
    QUANTITIES2 = {
        'Pressure,': press[i-i_init], # pressure
        'Temperature,': temp[i-i_init], # temperature
        'Salinity,': sal[i-i_init], # salinity
    }


    filenames = [f for f in os.listdir(DIR[i-i_init]) if (f.endswith('.cnv') or f.endswith('.ctd'))]
    filenames.sort()
    print(len(filenames))
    ### If files are in subdirectories (e.g.: 1db_down because 1db_up is also available)
    if len(filenames) == 0:
        DIRECTORY1 = [DIRECTORY1 for DIRECTORY1 in os.listdir(DIR[i-i_init]) if DIRECTORY1.endswith('down')]
        #print(globals()['DIRECTORY%s' %i],DIRECTORY1[0])
        DIR[i-i_init] = os.path.join(DIR[i-i_init],DIRECTORY1[0])
        print(DIR[i-i_init])
        filenames = [f for f in os.listdir(DIR[i-i_init]) if (f.endswith('.cnv') or f.endswith('.ctd'))]
        print(len(filenames))
    
    l=len(filenames)
    print(l)
    
    globals()['latitude%s' %i]=np.zeros(l)
    globals()['longitude%s' %i]=np.zeros(l)
    globals()['time%s' %i]=np.zeros(l, 'object')
    globals()['p%s' %i]=np.empty((l,5000))
    globals()['T%s' %i]=np.empty((l,5000))
    globals()['S%s' %i]=np.empty((l,5000))
    globals()['theta%s' %i]=np.empty((l,5000))
    globals()['dens%s' %i]=np.empty((l,5000))
    globals()['p%s' %i][:]=np.NAN    
    globals()['T%s' %i][:]=np.NAN
    globals()['S%s' %i][:]=np.NAN    
    globals()['theta%s' %i][:]=np.NAN
    globals()['dens%s' %i][:]=np.NAN
    print(globals()['latitude%s' %i].shape)
    print(globals()['S%s' %i].shape)

    for filename in sorted(filenames):
        f = open(os.path.join(DIR[i-i_init], filename), 'rt')
        j=-1
        ind=ind+1
        #print(f.shape[0][1][2)
        for line in f:
            line = line.strip()
            word = line.split()
            #print(line.split(',')[0])
            #print(word[0])
            #print(line[:])

            if (line.startswith('*') or word[0].startswith('L') or word[0].startswith('D')): #'*' for .cnv files; 'L' or 'D' 
#for .ctd files
                # header line, look for useful info
                if 'Latitude' in line:
                    #print(word[2])#,word[4],word[5])
                    if word[2] == 'Latitude':
                        globals()['latitude%s' %i][ind-1]= float(word[4])+ float(word[5])/60
                        #print(globals()['latitude%s' %i].shape)
                        #print(globals()['latitude%s' %i][0])
                    else:
                        globals()['latitude%s' %i][ind-1]= float(word[2])+ float(word[3])/60
                if 'LATITUDE' in line:
                    #print(word[2])
                    globals()['latitude%s' %i][ind-1]= float(word[2])
                if 'Longitude' in line:
                    #print(word[2],word[4],word[5])
                    if word[2] == 'Longitude':
                        globals()['longitude%s' %i][ind-1] = -(float(word[4])+ float(word[5])/60)                   
                    else:
                        globals()['longitude%s' %i][ind-1]= -float(word[2])+ float(word[3])/60
                if 'LONGITUDE' in line:
                    #print(word[2])
                    globals()['longitude%s' %i][ind-1]= -float(word[2])
                if '(Time)' in line:
                    #print(word[3])#,word[6],word[7],word[8])
                    if word[3] == '(Time)':
                        string = word[7] + ' ' + word[5] + ' ' + word[6] # Date only
                        #string = word[7] + ' ' + word[5] + ' ' + word[6] + ' ' + word[8] # Date & time
                        #print(string)
                        globals()['time%s' %i][ind-1] = datetime.datetime.strptime(str(string),'%Y %b %d') # Date only
                        #globals()['time%s' %i] = datetime.datetime.strptime(str(string),'%Y %b %d %H:%M:%S') # Date & time
                        #print(globals()['time%s' %i])
                    else:
                        string = word[5] + ' ' + word[3] + ' ' + word[4]
                        globals()['time%s' %i][ind-1] = datetime.datetime.strptime(str(string),'%Y %b %d') # Date only
                if 'DATE' in line:
                    #print(word[2])
                    string = word[2]
                    globals()['time%s' %i][ind-1] = datetime.datetime.strptime(str(string),'%d-%b-%Y')
                    globals()['time%s' %i][ind-1].strftime('%Y %b %d')
                if '** Station' in line:
                    station = line.partition(': ')[2]
                #if line == '*END*':
                 #   break
            elif line.startswith('# name'):     
                # comment line, look for plotable quantities information
                #print(QUANTITIES.items())
                #print(line.partition(': ')[2])
                #print(int(line.split(' ')[2]))
                if 'prDM' in line:
                    indexp = int(line.split(' ')[2])
                if 't090C' in line:
                    indexT = int(line.split(' ')[2])
                if 'sal00' in line:
                    indexS = int(line.split(' ')[2])
                if 'potemp090C' in line:
                    indexTheta = int(line.split(' ')[2])
                if 'sigma-' and '00' in line: #sigma-é00 but troubles to read "é"
                    indexSig = int(line.split(' ')[2])
                    #print(indexS)
                
                
            elif (line.startswith('1') or line.startswith('2') or line.startswith('3') 
                  or line.startswith('4') or line.startswith('5') or line.startswith('6') or 
                  line.startswith('7') or line.startswith('8') or line.startswith('9')) and ',' in line: ### ctd files 
                    j=j+1
                    #print(line.split(',')[2])
                    globals()['p%s' %i][ind-1,j]= line.split(',')[0]
                    globals()['T%s' %i][ind-1,j]= line.split(',')[1]
                    globals()['S%s' %i][ind-1,j]= line.split(',')[2]

            elif (line.startswith('1') or line.startswith('2') or line.startswith('3') 
                  or line.startswith('4') or line.startswith('5') or line.startswith('6') or 
                  line.startswith('7') or line.startswith('8') or line.startswith('9')) and ',' not in line: ### CNV files
                    j=j+1                    
                    globals()['p%s' %i][ind-1,j]= word[indexp]
                    globals()['T%s' %i][ind-1,j]= word[indexT]
                    globals()['S%s' %i][ind-1,j]= word[indexS]
                    if float(word[indexSig]) < 15.0: ### mistake in files from 2004: inversion of density column and pot temp column 
                        #print("Inversion Theta/Sig!!!")
                        globals()['theta%s' %i][ind-1,j]= word[indexSig]
                        globals()['dens%s' %i][ind-1,j]= word[indexTheta]
                    else:
                        globals()['theta%s' %i][ind-1,j]= word[indexTheta]
                        globals()['dens%s' %i][ind-1,j]= word[indexSig]    
                    #print(globals()['dens%s' %i][ind-1,j])
                    
    #print(globals()['longitude%s' %i][2],globals()['time%s' %i])
    #print(globals()['S%s' %i][0,:],globals()['S%s' %i][1,:])
    #print(globals()['S%s' %i][0,:],globals()['S%s' %i][1,:])


## Print the variables that were extracted from the files
for i in xrange(i_init, i_final+1,1):
    print (i,globals()['p%s' %i][0,0],globals()['T%s' %i][0,0],globals()['S%s' %i][0,0],globals()['theta%s' %i][0,0],
           globals()['dens%s' %i][0,0])
    ## ==> No theta2003 and dens2003
    

plt.show()
""""""

## Save the variables p2003-p2013, T2003-T2013, S2003-S2013, theta2003-theta2013, and dens2003-dens2013 
## into a file ... .pickle

import pickle
yr=list(np.chararray((i_final-i_init+1)))


## Saving the objects:
for i in xrange(i_init, i_final+1,1):
    k=i-i_init
    globals()['yr%s' %k]=i
print(yr0,yr2)
#print(yr0,globals()['longitude%s' %yr0][0],globals()['latitude%s' %yr0][0],globals()['time%s' %yr0][0],
#      globals()['p%s' %yr0][0,0],globals()['T%s' %yr0][0,0],globals()['S%s' %yr0][0,0],globals()['theta%s' %yr0][0,0],
#      globals()['dens%s' %yr0][0,0])

"""
with open('CandaBasinWHOI-CTDdata.pickle', 'wb') as fp2:
    pickle.dump([globals()['longitude%s' %yr0],globals()['longitude%s' %yr1],globals()['longitude%s' %yr2],
                 globals()['longitude%s' %yr3],globals()['longitude%s' %yr4],globals()['longitude%s' %yr5],
                 globals()['longitude%s' %yr6],globals()['longitude%s' %yr7],globals()['longitude%s' %yr8],
                 globals()['longitude%s' %yr9],globals()['longitude%s' %yr10],
                 globals()['latitude%s' %yr0],globals()['latitude%s' %yr1],globals()['latitude%s' %yr2],
                 globals()['latitude%s' %yr3],globals()['latitude%s' %yr4],globals()['latitude%s' %yr5],
                 globals()['latitude%s' %yr6],globals()['latitude%s' %yr7],globals()['latitude%s' %yr8],
                 globals()['latitude%s' %yr9],globals()['latitude%s' %yr10],
                 globals()['time%s' %yr0],globals()['time%s' %yr1],globals()['time%s' %yr2],
                 globals()['time%s' %yr3],globals()['time%s' %yr4],globals()['time%s' %yr5],
                 globals()['time%s' %yr6],globals()['time%s' %yr7],globals()['time%s' %yr8],
                 globals()['time%s' %yr9],globals()['time%s' %yr10],
                 globals()['p%s' %yr0],globals()['p%s' %yr1],globals()['p%s' %yr2],globals()['p%s' %yr3],
                 globals()['p%s' %yr4],globals()['p%s' %yr5],globals()['p%s' %yr6],globals()['p%s' %yr7],
                 globals()['p%s' %yr8],globals()['p%s' %yr9],globals()['p%s' %yr10],
                 globals()['T%s' %yr0],globals()['T%s' %yr1],globals()['T%s' %yr2],globals()['T%s' %yr3],
                 globals()['T%s' %yr4],globals()['T%s' %yr5],globals()['T%s' %yr6],globals()['T%s' %yr7],
                 globals()['T%s' %yr8],globals()['T%s' %yr9],globals()['T%s' %yr10],
                 globals()['S%s' %yr0],globals()['S%s' %yr1],globals()['S%s' %yr2],globals()['S%s' %yr3],
                 globals()['S%s' %yr4],globals()['S%s' %yr5],globals()['S%s' %yr6],globals()['S%s' %yr7],
                 globals()['S%s' %yr8],globals()['S%s' %yr9],globals()['S%s' %yr10],
                 globals()['theta%s' %yr0],globals()['theta%s' %yr1],globals()['theta%s' %yr2],globals()['theta%s' %yr3],
                 globals()['theta%s' %yr4],globals()['theta%s' %yr5],globals()['theta%s' %yr6],globals()['theta%s' %yr7],
                 globals()['theta%s' %yr8],globals()['theta%s' %yr9],globals()['theta%s' %yr10],
                 globals()['dens%s' %yr0],globals()['dens%s' %yr1],globals()['dens%s' %yr2],globals()['dens%s' %yr3],
                 globals()['dens%s' %yr4],globals()['dens%s' %yr5],globals()['dens%s' %yr6],globals()['dens%s' %yr7],
                 globals()['dens%s' %yr8],globals()['dens%s' %yr9],globals()['dens%s' %yr10]], fp2)
"""

2003
0
/home/mgrenier/Documents/Data_Arctic/CTDBeaufortGyreWHOI/LSSL_ctd2003/1db_down
47
47
(47,)
(47, 5000)
2004
50
50
(50,)
(50, 5000)
2005
0
/home/mgrenier/Documents/Data_Arctic/CTDBeaufortGyreWHOI/LSSL_ctd2005/1db_down
50
50
(50,)
(50, 5000)
2006
0
/home/mgrenier/Documents/Data_Arctic/CTDBeaufortGyreWHOI/LSSL_ctd2006/1db_down
64
64
(64,)
(64, 5000)
2007
0
/home/mgrenier/Documents/Data_Arctic/CTDBeaufortGyreWHOI/LSSL_ctd2007/1db_down
106
106
(106,)
(106, 5000)
2008
146
146
(146,)
(146, 5000)
2009
53
53
(53,)
(53, 5000)
2010
72
72
(72,)
(72, 5000)
2011
52
52
(52,)
(52, 5000)
2012
56
56
(56,)
(56, 5000)
2013
55
55
(55,)
(55, 5000)
2003 1.0 -1.1999 28.4279 nan nan
2004 1.0 1.797 31.0125 1.797 24.7918
2005 1.0 -0.0107 28.9924 -0.0107 23.2594
2006 2.0 -0.7354 26.5352 -0.7354 21.2939
2007 2.0 8.2284 29.3919 8.2282 22.8415
2008 1.0 7.4125 29.5225 7.4124 23.0543
2009 1.0 4.232 27.8397 4.2319 22.0754
2010 3.0 4.8429 29.7554 4.8427 23.5361
2011 1.0 2.2528 24.9825 2.2528 19.9422
2012 2.0 7.622

"\nwith open('CandaBasinWHOI-CTDdata.pickle', 'wb') as fp2:\n    pickle.dump([globals()['longitude%s' %yr0],globals()['longitude%s' %yr1],globals()['longitude%s' %yr2],\n                 globals()['longitude%s' %yr3],globals()['longitude%s' %yr4],globals()['longitude%s' %yr5],\n                 globals()['longitude%s' %yr6],globals()['longitude%s' %yr7],globals()['longitude%s' %yr8],\n                 globals()['longitude%s' %yr9],globals()['longitude%s' %yr10],\n                 globals()['latitude%s' %yr0],globals()['latitude%s' %yr1],globals()['latitude%s' %yr2],\n                 globals()['latitude%s' %yr3],globals()['latitude%s' %yr4],globals()['latitude%s' %yr5],\n                 globals()['latitude%s' %yr6],globals()['latitude%s' %yr7],globals()['latitude%s' %yr8],\n                 globals()['latitude%s' %yr9],globals()['latitude%s' %yr10],\n                 globals()['time%s' %yr0],globals()['time%s' %yr1],globals()['time%s' %yr2],\n                 globals()['time%s' %yr3],